# 7.4 도로명주소 유효성 평가 및 정제
---
- 최종 수정일 : 2024-03
- **7.1.에서 생성한 .env 파일도 로드해주세요**
- 정제가 완료된 파일들은 refine 폴더에 있습니다.

In [1]:
!pip install python-dotenv
!pip install plotly

In [2]:
import pandas as pd
import numpy as np

import warnings
# 경고 메시지 숨기기
warnings.filterwarnings(action='ignore')

import re
import requests

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from dotenv import load_dotenv
import os
from tqdm import tqdm
load_dotenv(".env")

API_ID = os.getenv("CLIENT_ID")
API_SECRET = os.getenv("CLIENT_SECRET")

import json

## 0. 필요 데이터, 함수 준비

- 본 문서에서 필요한 데이터
    - 깃헙 : https://github.com/hike-lab/address-data-guide/tree/main/chapter-7/data
    - 구글 드라이브 : 

In [3]:
df = pd.read_csv("sample.csv", encoding='cp949')
# 결측치의 데이터 타입을 None으로 변경
df = df.replace(np.nan, '-').replace("-",None)

sido_map = json.load(open("sido_mapping_table.json", "r"))
sido_map = sido_map["data"]

### 0.1 평가 테이블 준비

개별 값에 대한 평가값이 입력될 평가 테이블을 생성한다. 각 컬럼이 의미하는 바는 아래와 같다.

- pattern : 구문 오류
- unvaild_city : 생략 자치단체명
- exist : 주소의 실존 여부
- API_addr : 동/읍/면 명칭 추가
- data : 원본 주소

In [4]:
datalength = len(df)
addr_error = pd.DataFrame(np.zeros((datalength, 4)),columns=["pattern","unvalid_city","exist", "API_addr"])
addr_error["data"] = df['소재지도로명주소']
addr_error

,pattern,unvalid_city,exist,API_addr,data
0,0.0,0.0,0.0,0.0,강원도 평창군 평창읍 평창중앙로 67
1,0.0,0.0,0.0,0.0,강원도 평창군 미탄면 청옥산1길 3-9
2,0.0,0.0,0.0,0.0,강원도 평창군 피탄면 청옥산1길 13
3,0.0,0.0,0.0,0.0,강원도 평창군 방림면 서동로 1338-3
4,0.0,0.0,0.0,0.0,강원도 평창군 방림면 계촌길 101
...,...,...,...,...,...
494,0.0,0.0,0.0,0.0,충청남도 예산군 봉산면 봉산로 420
495,0.0,0.0,0.0,0.0,충청남도 예산군 봉산면 봉산로 511
496,0.0,0.0,0.0,0.0,충청남도 예산군 봉산면 금치1길 9
497,0.0,0.0,0.0,0.0,충청남도 예산군 봉산면 옹안길 37


### 0.2 시각화 함수 준비

In [5]:
def make_error_rate_plot(addr_error, col):
    # colors
    # 정상 : darkblue, 오류 : darkred, 공백 : lightgrey
    COLORS = ["darkblue", "darkred", "lightgrey"]

    # 오류 비중 계산

    errors = addr_error[col].value_counts()[1]
    normal = addr_error[col].value_counts()[0]
    blank = len(df) - errors - normal

    # 서브플롯 객체 생성
    # 히트맵을 위해 2개 컬럼에 걸쳐 히트맵 plot 배정
    fig = make_subplots(rows=1, cols=3,
                        specs=[[{'type':'pie'}, {"colspan": 2}, None]],
                        subplot_titles=("오류 데이터 비율","오류 행 현황"),
                        horizontal_spacing = 0.12)

    # 파이차트 추가
    fig.add_trace(go.Pie(labels=['정상','오류','공백'], textinfo='label+percent',
                        values=[normal, errors, blank],marker=dict(colors=COLORS), showlegend=False,),
                row=1, col=1)
    # 히트맵차트 추가
    fig.add_trace(go.Heatmap(z=[addr_error[col]], x = list(addr_error.index), y= ["row#"],
                            colorscale = [[0, "darkblue"],[1, "darkred"]]), row=1, col=2)

    fig.show()
    return fig

## 1. 오류유형 패턴을 통한 유효성 평가 & 정제

**동일패턴**
- pattern 1 : 특별/광역시/특별자치도/도 + 자치시/행정시/구 + 도로명
- pattern 2 : 광역시/도/특별자치도 + 구/군/자치시 + 읍/면 + 도로명
- pattern 3 : 특별자치시 + 도로명
- pattern 4 : 특별자치시 + 읍/면 + 도로명
- pattern 5 : 도 + 자치시 + 일반구 + 도로명
- pattern 6 : 도 + 자치시 + 일반구 + 읍/면 + 도로명

In [6]:
# 특별/광역시/특별자치도/도 + 구/자치시/행정시 + 도로명 + 건물번호
pattern1 = r"(?P<province>[가-힣]+)(?:특별시|광역시|특별자치도|도)\s+(?P<city>[가-힣]+)(시|구)\s+(?P<road>[가-힣\d]+)(?:로|길)\s+(?P<number>[\d])"

# 광역시/도/특별자치도 + 자치구/군 + 읍/면 + 도로명 + 건물번호
pattern2 = r"(?P<province>[가-힣]+)(?:도|광역시|특별자치도)\s+(?P<city>[가-힣]+)(시|군|구)\s+(?P<local>[가-힣]+)(읍|면)\s+(?P<road>[가-힣\d]+)(?:로|길)\s+(?P<number>[\d])"

# 특별자치시 + 읍/면 + 도로명 + 건물번호
pattern3 = r"(?P<province>[가-힣]+)(?:특별자치시)\s+(?P<type>[가-힣]+)(?:읍|면)\s+(?P<road>[가-힣\d]+)(?:로|길)\s+(?P<number>[\d])"

# 특별자치시 + 도로명 + 건물번호
pattern4 = r"(?P<province>[가-힣]+)(?:특별자치시)\s+(?P<road>[가-힣\d]+)(?:로|길)\s+(?P<number>[\d])"

# 특별자치도/도 + 자치/행정시 + 일반구 + 도로명 + 건물번호
pattern5 = r"(?P<province>[가-힣]+)(?:도|특별자치도)\s+(?P<city>[가-힣]+)(?:시)\s+(?P<type>[가-힣]+)(?:구)\s+(?P<road>[가-힣\d]+)(?:로|길)\s+(?P<number>[\d])"

# 특별자치도/도 + 자치/행정시 + 일반구 + 읍/면 + 도로명 + 건물번호
pattern6 = r"(?P<province>[가-힣]+)(?:도|특별자치도)\s+(?P<city>[가-힣]+)(?:시)\s+(?P<type>[가-힣]+)(?:구)\s+(?P<local>[가-힣]+)(?:읍|면)\s+(?P<road>[가-힣\d]+)(?:로|길)\s+(?P<number>[\d])"


def element_check(i):
    item = df['소재지도로명주소'][i]
    if item != None:
        # 패턴을 통한 구문 오류 체크
        if re.match(pattern1, item) or re.match(pattern2, item) or re.match(pattern3, item) or re.match(pattern4, item) or re.match(pattern5, item) or re.match(pattern6, item):
            addr_error["pattern"][i] = 0

        else:
            addr_error["pattern"][i] = 1

        # 축약된 광역자치단체명 체크
        sido = df['소재지도로명주소'][i].split(" ")[0]
        if sido in sido_map.keys():
            addr_error["unvalid_city"][i] = 1
        else:
            addr_error["unvalid_city"][i] = 0
    # 공백값
    else:
        addr_error["pattern"][i] = None
        addr_error["unvalid_city"][i] = None

df.index.map(lambda i: element_check(i))

Index([None, None, None, None, None, None, None, None, None, None,
       ...
       None, None, None, None, None, None, None, None, None, None],
      dtype='object', length=499)

### 1.2. 시각화

In [7]:
add_error_fig = make_error_rate_plot(addr_error, "pattern")
add_error_fig.write_html("4-4-addr-error.html")

### 1.3. 구문 패턴에 따라 정제하기

#### 1.3.1. 문제가 있는 데이터만 추출
- 구문 오류가 있거나, 축약된 시도명을 포함하고 있는 데이터의 행만 추출한다.

In [8]:
errs = addr_error[(addr_error["pattern"] == 1) | ( addr_error["unvalid_city"] == 1)]

#### 1.3.2. 구문 오류 패턴 정제를 위한 패턴식 정의
- 오류 유형에 따른 패턴을 식별하고, 문제의 부분에 대한 위치값을 리턴할 수 있도록 정규표현식을 작성한다.

In [9]:
# 구문 오류 패턴식

# 도로명-건물번호가 붙어있는 경우
error_pat1 = r"(?P<road>[가-힣\d]+로)(?P<number>\d+(?:-\d+)?)"
error_pat2 = r"(?P<road>[가-힣\d]+로)(?P<number>\d+(?:-\d+)?)"
error_pat3 = r"(?P<road>[가-힣\d]+로)(?P<subroad>[가-힣\d]+번길)(?P<number>\d+(?:-\d+)?)"
error_pat3_ = r"(?P<road>[가-힣\d]+로)(?P<subroad>[가-힣\d]+길)(?P<number>\d+(?:-\d+)?)"
error_pat4 = r"(?P<road>[가-힣\d]+\d?길)(?P<number>\d+(?:-\d+)?)"

# 지하번호 기입 오류
error_pat5 = r"(?P<ground>지하)+\s(?P<number>\d+(?:-\d+)?)"

# 참고항목과 건물번호를 붙여 씀
error_pat6 = r"(?P<number>\d+(?:-\d+)?)\((?P<dong>.+?)\)"

# 도로명이 분리되어 있는 경우
error_pat7 = r"(?P<road>[가-힣\d]+로)+\s(?P<subroad>[가-힣\d]+번길)+\s(?P<number>\d+(?:-\d+)?)"
error_pat8 = r"(?P<road>[가-힣\d]+로)+\s(?P<subroad>[가-힣\d]+길)+\s(?P<number>\d+(?:-\d+)?)"
error_pat9 = r"(?P<road>[가-힣\d]+[^시군구도읍면동])+\s(?P<subroad>[가-힣\d]+[번길|길|로])+\s(?P<number>\d+(?:-\d+)?(?:\d+)?)"

# 동층호 별도 표기
error_pat10 = r"(?P<det>(?:\d+층|[가-힣a-zA-Z\d]동+(?:\s)\d층+(?:\s)\d호|[가-힣a-zA-Z\d]동+(?:\s)\d호|\d호|\d층+(?:\s)\d호))"
error_pat11 =  r"\((?P<dong>.+?)\)+(?:,|,\s|\s|\s,|\s,\s)(?P<det>(?:\d+층|[가-힣a-zA-Z\d]동+(?:\s)\d층+(?:\s)\d호|[가-힣a-zA-Z\d]동+(?:\s)\d호|\d호|\d층+(?:\s)\d호))"

# 동 끝에 지번까지 표기된 경우
error_pat12 =r"\((?P<dong>.+)동(?P<number>\s?\d+)(?:-\d+)?\)"

#### 1.3.3. 정제에 필요한 함수 정의

- `get_replace` : 문제의 패턴이 존재하는 부분의 문자열을 리턴한다.

In [10]:
def get_replace(pt, addr):
    idx_s = pt.span()[0]
    idx_e = pt.span()[1]
    replacement = addr[idx_s:idx_e]
    return replacement


- `not_comp_all` : 건물명이 참고사항이 아닌 주소의 구성요소로 들어가 있는 경우를 감지하고, 건물명을 제거한 주소를 리턴한다.
    
    **내장함수**
    - `not_comp` : 주소 내부에 건물명이 포함된 경우를 조사한다.
    - `square_lambd` : map 메서드를 통해 활용하기 위해 `not_comp`를 lambda값에 적용시키는 함수이다.

In [11]:
def not_comp(word):
    if word.isalpha()==True and word[-1] not in ["도","시","군","구","로","대로","길","층","동","호",'읍','면'] and "지하" not in word and "(" not in word  and ")" not in word  and "-" not in word and "," not in word:
        return "error"
    else:
        return word

square_lambda = lambda x: not_comp(x)

def not_comp_all(addr):
    addr_list = addr.split(" ")
    addr_li = list(map(square_lambda, addr_list))
    new_add = " ".join(addr_li)
    new_add = new_add.replace("error", "").replace("  ", " ")
    return new_add

- `add_refine` : 앞서 선언한 패턴과 함수를 단계적으로 적용해 주소를 정제하는 함수이다

    **세부 정제 단계**
        
    1. 중복 요소 제거
        
    2. 광역자치단체명 생략 주소 정제

    3. 도로명과 건물번호의 오류유형별 정제

        3.1. 도로명과 건물번호가 붙어 있는 경우

        3.2. 지하번호가 띄어쓰기로 기입된 경우

        3.3. 참고항목과 건물번호를 붙여 쓴 경우

        3.4. 하나의 도로명이 분리되어 기재된 경우
            
    4. 참고항목으로 기입되지 않은 건물명 삭제

    5. 상세주소와 참고항목에서의 오류유형별 정제

        5.1. 참고항목이 상세주소에 앞서 기재된 경우

        5.2. 참고항목의 동명칭에 지번까지 기입된 경우

In [12]:

def add_refine(addr):
    # 기본 구문 오류 : 동일한 요소가 중복되어 작성된 경우
    addr = re.sub(r"(?P<word>\w+)\s(?P=word)\s", r"\1 ", addr, count=1)
    sido =addr.split(" ")[0]

    # 광역자치단체 단위 생략한 경우 정제
    if sido in sido_map.keys():
        addr = addr.replace(sido, sido_map[sido])
    else:
        pass

    # 도로명-건물번호가 붙어있는 경우
    p1 = re.search(error_pat1, addr)
    if p1 != None:
        repl = get_replace(p1, addr)
        newadd= p1["road"] + " " + p1["number"]
        addr = addr.replace(repl, newadd)
    p2 = re.search(error_pat2, addr)
    if p2 != None:
        repl = get_replace(p2, addr)
        newadd= p2["road"] + " " + p2["number"]
        addr = addr.replace(repl, newadd)
    p3 = re.search(error_pat3, addr)
    if p3 != None:
        repl = get_replace(p3, addr)
        newadd= p3["road"] + p3["subroad"] + " " + p3["number"]
        addr = addr.replace(repl, newadd)
    p3_ = re.search(error_pat3_, addr)
    if p3_ != None:
        repl = get_replace(p3_, addr)
        newadd= p3_["road"] + p3_["subroad"] + " " + p3_["number"]
        addr = addr.replace(repl, newadd)
    p4 = re.search(error_pat4, addr)
    if p4 != None:
        repl = get_replace(p4, addr)
        newadd= p4["road"] + " " + p4["number"]
        addr = addr.replace(repl, newadd)

    # 지하번호 기입 오류
    p5 = re.search(error_pat5, addr)
    if p5 != None:
        repl = get_replace(p5, addr)
        newadd= "지하" + p5["number"]
        addr = addr.replace(repl, newadd)

    # 참고항목과 건물번호를 붙여 씀
    p6 = re.search(error_pat6, addr)
    if p6 != None:
        repl = get_replace(p6, addr)
        newadd= p6["number"] + " "+"(" + p6["dong"] + ")"
        addr = addr.replace(repl, newadd)

    # 도로명이 분리되어 있는 경우
    p7 = re.search(error_pat7, addr)
    if p7 != None:
        repl = p7["road"] + " " +p7["subroad"] + " " + p7["number"]
        newadd= p7["road"] + p7["subroad"] + " " + p7["number"]
        addr = addr.replace(repl, newadd)

    p8 = re.search(error_pat8, addr)
    if p8 != None:
        repl = p8["road"] + " " +p8["subroad"] + " " + p8["number"]
        newadd= p8["road"] + p8["subroad"] + " " + p8["number"]
        addr = addr.replace(repl, newadd)

    p9 = re.search(error_pat9, addr)
    if p9 != None:
        repl = p9["road"] + " " +p9["subroad"] + " " + p9["number"]
        newadd= p9["road"] + p9["subroad"] + " " + p9["number"]
        addr = addr.replace(repl, newadd)

    # 병기된 건물명 삭제
    addr = not_comp_all(addr)

    # 참고항목이 상세주소보다 앞에 나오는 경우 - 1
    p10 = re.search(error_pat10, addr)
    if p10 != None:
        remove_pos_start = p10.span()[0]
        remove_pos_end = p10.span()[1]
        remove_string = addr[remove_pos_start:remove_pos_end]
        addr = addr.replace(remove_string, "")
        addr = addr +  ", " + p10["det"]

    # 참고항목이 상세주소보다 앞에 나오는 경우 - 2
    p11 = re.search(error_pat11, addr)
    if p11 != None:
        remove_pos_start = p11.span()[0]
        remove_pos_end = p11.span()[1]
        remove_string = addr[remove_pos_start:remove_pos_end]
        addr = addr.replace(remove_string, "")
        addr = addr +  ", " + p11["det"] + " " + "(%s)"%p11["dong"]

    # 동 끝에 지번까지 표기된 경우
    p12 = re.search(error_pat12, addr)
    if p12 != None:
        repl = get_replace(p12, addr)
        new_detail = "(%s동)"%p12["dong"]
        addr = addr.replace(repl, new_detail)


    return addr


# 정규표현식 패턴에 따라 주소를 정제하는 Lambda 함수를 적용
newadds = errs['data'].apply(lambda x: add_refine(x))


### 1.3.4. 함수를 적용하여 정제

In [13]:
# 오류 데이터에 대해 정제
newadds = errs['data'].apply(lambda x: add_refine(x))

# 오류가 있는 행에 대해서 정제된 새 주소 대체
df['소재지도로명주소'][errs.index] = newadds

# 정제 전후의 비교를 위해 errs 테이블에도 저장
errs['revised'] = newadds

### 1.4. 정제 후 재확인

In [14]:
df.index.map(lambda i: element_check(i))
add_error_fig = make_error_rate_plot(addr_error, "pattern")
#add_error_fig.write_html("6-4-addr-after-refine.html")

## 2. API를 통한 정확성 평가 & 정제

이번에는 네이버 API를 활용해 도로명주소가 실존하는 주소인지 확인해봅시다.

### 2.1. 필요 함수 준비

- `search_addr` : 주소를 네이버 Geocode API에 검색하는 함수

In [15]:
# 주소 검색 함수
def search_addr(addr):
    # 요청 헤더에는 API 키와 아이디 값을 입력합니다.
    headers = {"X-NCP-APIGW-API-KEY-ID":API_ID, "X-NCP-APIGW-API-KEY":API_SECRET}

    # 파라미터에는 검색할 주소를 입력합니다.
    params = {"query" : addr, "output":"json"}

    # 정보를 요청할 url입니다
    url ="https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"

    data = requests.get(url, headers=headers, params=params)

    return json.loads(data.text)


### 2.2. API를 통한 도로명주소 실존 여부 검사

API를 통해 주소의 실존 여부를 검색하고, 만일 주소가 실존하는 주소일 경우, 추후 인구데이터와의 결합을 위해 법정동정보를 따로 저장합니다.

In [16]:
for i in tqdm(range(len(df))):
    if df['소재지도로명주소'][i] != None:
        re_val = search_addr(df.loc[i, "소재지도로명주소"])

        # 검색한 주소가 존재하는지 확인
        if (re_val["status"] != "OK") or (re_val["meta"]["totalCount"] == 0):
            # 주소가 검색되지 않는 경우
            addr_error['exist'][i] = 1
            addr_error['API_addr'][i] = None

        else:
            if re_val["status"] == "OK":
                # 주소가 검색되는 경우
                addr_error['exist'][i] = 0

                # 동명칭을 추가합니다.
                dong = re_val['addresses'][0]["addressElements"][2]["longName"]
                addr_error['API_addr'][i] = dong
    else:
        addr_error['exist'][i] = None
        addr_error['API_addr'][i] = None


100%|██████████| 499/499 [08:35<00:00,  1.03s/it]


시각화를 통해 실제로 존재하는 주소의 비중이 얼마나 되는지 확인해봅시다. 앞서 사용하였던 `make_error_rate_plot`을 다시 활용합니다.

In [17]:
add_error_fig = make_error_rate_plot(addr_error, "exist")
add_error_fig.write_html("4-4-addr-exist.html")

이미 한 차례 구문오류에 대한 정제도 끝났기에, 실존하지 않는 주소의 비율이 그렇게 높게 나타나지 않은 것으로 보입니다.

### 2.2. 좌표값을 활용한 정제

이번에는 실존하지 않는 주소에 대해, 좌표값을 검색하여 리턴되는 주소로 대체해줍시다. 이때, 비어있는 데이터도 함께 채워줍니다.

In [18]:
idx = addr_error[addr_error["exist"]==1].index.tolist()
idx += addr_error[addr_error["exist"].isnull()].index.tolist()

#### 2.2.1. 필요 함수 준비

- `search_coords` : 좌표계를 네이버 reverse geocoding API에 검색하는 함수입니다. '경도, 위도' 순으로 입력해주세요.

In [19]:
# 경도, 위도 순으로 입력
def search_coords(x,y):
    coord = f"{x},{y}"
    # 요청 헤더에는 API 키와 아이디 값을 입력합니다.
    headers = {"X-NCP-APIGW-API-KEY-ID":API_ID, "X-NCP-APIGW-API-KEY":API_SECRET}

    # 파라미터에는 변환할 좌표계를 입력합니다. "경도,위도" 순으로 입력해주세요.
    params = {"coords" : coord, "output":"json", "orders":"roadaddr,addr"}

    # 정보를 요청할 url입니다
    url ="https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc"

    data = requests.get(url, headers=headers, params=params)

    return json.loads(data.text)

- `road_addr_maker` : `search_coords`에서 리턴된 주소 구성 요소들을 합성하여 새 도로명주소를 생성하는 함수입니다.

In [20]:
# 도로명주소를 합성하는 함수
def road_addr_maker(road_obj):
    road = road_obj["region"]["area1"]["name"] + " " + road_obj["region"]["area2"]["name"]
    if road_obj["region"]["area3"]["name"][-1] == "읍" or road_obj["region"]["area3"]["name"][-1] == "면":
        road += " " + road_obj["region"]["area3"]["name"]
    if road_obj["land"]["name"] != "":
        road += " " + road_obj["land"]["name"]
    if road_obj["land"]["number1"] != "":
        road += " " + road_obj["land"]["number1"]
    if  road_obj["land"]["number2"] != "":
        road += "-" + road_obj["land"]["number2"]
    return road


### 2.2.2. 함수 실행

API를 활용하는 경우, 서버에 하나의 요청만 보낼 수 있으므로, 반복문을 사용해 한 행씩 실행시켜야 합니다. 또한, 서버와 통신하여 결과값을 가져오므로 다른 과정들보다 시간이 오래 걸립니다.

In [21]:
# 좌표계 데이터 역시 잘못된 경우 저장
non_exist_coord = []

for i in tqdm(idx):
        # 좌표계 검색
        re_val = search_coords(df.loc[i, "경도"],df.loc[i, "위도"])

        # 검색한 좌표계가 존재하는지 확인
        if (re_val["status"]["name"] != "ok") or (len(re_val["results"]) == 0):
            non_exist_coord.append(i)

        # 검색한 좌표계가 존재하는 경우
        else:
            # 지번, 도로명주소가 모두 존재하는 지점인 경우
            if len(re_val["results"]) == 2:
                road_obj = re_val["results"][0]

                # 주소 합성
                road = road_addr_maker(road_obj)
                df['소재지도로명주소'][i] = road
                addr_error['API_addr'][i] = road_obj["region"]["area3"]["name"]

100%|██████████| 40/40 [00:43<00:00,  1.09s/it]


좌표계 역시 오류가 있는지 확인합니다. 좌표계조차 오류가 있어 검색이 되지 않는 경우, 해당 주소를 정제하는 것이 불가능합니다.

In [22]:
non_exist_coord

[]

지금까지 정제한 데이터를 저장합니다. 에러테이블도 함께 저장하겠습니다.

In [23]:
df.to_excel("sample_refined.xlsx")
addr_error.to_excel("error_refine_by_API.xlsx", index=False)

## 3. 인구 데이터와 합치기

이제 법정동별 인구정보 데이터와 샘플 데이터를 병합해보도록 하겠습니다. 인구데이터는 [사이트]()에서 제공합니다. 사이트의 집계기준은 행정동이지만, 본 문서에서 활용할 데이터는 이를 법정동으로 변환하여 재합산한 데이터를 활용합니다. 이 과정에서 실행한 코드도 [깃허브]()에 데이터 원본과 함께 업로드 해두었습니다.

### 3.1. 인구데이터의 행정기관 단위로 행정기관명칭 수정

도심지역이 아닌 도서산간 지역의 경우, 행정동보다 큰 단위인 법정동 단위로도 집계하여도 인구수가 너무 적어, 복수의 법정동을 합산하여 집계하고 있습니다. 따라서, 본 문서에서도 다른 행정구역과 합산되는 법정동/읍/면의 경우 인구데이터에 기재된대로 수정합니다.

In [24]:
census = pd.read_excel("census_data_cleaned.xlsx") # 인구 데이터
merge_table = json.load(open("dong_merge_table.json", "r")) # 동 명칭 병합이 필요한 테이블
merge_table = merge_table["data"]

# /읍/면/동명칭의 수정이 필요한 지역들에 대한 리스트
change_list = list(merge_table.keys())

- `merging_dong` : 수정이 필요한 행정기관 명칭인지 파악하고, merge_table의 값대로 수정합니다.

In [25]:
def mergeing_dong(i):
    if df['소재지도로명주소'][i] != None:
        addr_splits = df['소재지도로명주소'][i].split(" ")

        city = addr_splits[0] + " " + addr_splits[1]
        if addr_error["API_addr"][i] in change_list and city == merge_table[addr_error["API_addr"][i]]["city"]:
            addr_error["API_addr"][i] = merge_table[addr_error["API_addr"][i]]["changeto"]
addr_error.index.map(lambda i: mergeing_dong(i))

Index([None, None, None, None, None, None, None, None, None, None,
       ...
       None, None, None, None, None, None, None, None, None, None],
      dtype='object', length=499)

### 3.2. 행정기관 명칭을 키 값으로 하여 병합하기

앞서 샘플 데이터의 읍/면/동 명칭을 모두 수정하였습니다. 그럼 이제 이 행정기관 명칭을 기준으로 두 데이터를 병합해봅시다.

In [26]:
census["key"] = census['행정기관']
df["동명칭"] = addr_error["API_addr"]

- `key_make` : 샘플 데이터의 "동명칭" 컬럼은 단순히 동 단위의 명칭만 담고 있습니다. 동명칭의 경우, 중복되는 경우가 많으므로, 인구데이터의 "행정기관" 컬럼과 같이 **광역자치단체명+(기초자치단체명)+동/읍/면** 의 형태로 데이터를 보충하여 키 컬럼을 따로 만듭니다.

In [32]:
def key_make(i):
    if df['소재지도로명주소'][i] != None:
        key_match = re.search(r"([가-힣]+(?:특별시|광역시|특별자치도|도|특별자치시))(\s[가-힣]+(?:시|군|구))?(\s[가-힣]+(?:구))?", df['소재지도로명주소'][i])
        if key_match != None and type(df['동명칭'][i]) == str:
            return key_match.group() + " " + df["동명칭"][i]
        else:
            return None
    else:
        if df['소재지지번주소'][i] != None:
            key_match = re.search(r"([가-힣]+(?:특별시|광역시|특별자치도|도|특별자치시))(\s[가-힣]+(?:시|군|구))?(\s[가-힣]+(?:구))?", df['소재지지번주소'][i])
            if key_match != None and type(df['동명칭'][i]) == str:
                return key_match.group() + " " + df["동명칭"][i]
            else:
                return None

# 병합의 기준이 되는 키 컬럼을 생성합니다
df.reset_index(inplace=True, drop = True)
df["key"] = df.index.map(lambda i: key_make(i))

In [33]:
# key 컬럼을 기준으로 샘플데이터에 인구 데이터를 병합합니다.
merged = pd.merge(left = df, right = census, on="key", how="left")

In [34]:
merged.head(5)

,개방시설명,휴관일,평일운영시작시각,평일운영종료시각,주말운영시작시각,주말운영종료시각,유료사용여부,사용기준시간,사용료,초과사용단위시간,...,경도,데이터기준일자,동명칭_x,key,행정기관,총인구수,세대수,세대당 인구,남여 비율,동명칭_y
0,평창군문화복지센터,공휴일+일요일,9:00,22:00,9:00,18:00,Y,None,"강당/다목적강당:80000(오전)+80000(오후)+100000(야간)+150,00...",None,...,128.394885,2022-11-30,평창읍,강원특별자치도 평창군 평창읍,강원특별자치도 평창군 평창읍,8356.0,4304.0,1.94,1.06,평창읍
1,미탄복지회관,연중무휴,9:00,18:00,9:00,18:00,Y,2.0,40000,None,...,128.495722,2022-11-30,미탄면,강원특별자치도 평창군 미탄면,강원특별자치도 평창군 미탄면,1563.0,919.0,1.70,1.19,미탄면
2,미탄복지회관,연중무휴,9:00,18:00,9:00,18:00,Y,4.0,30000,None,...,128.496254,2022-11-30,미탄면,강원특별자치도 평창군 미탄면,강원특별자치도 평창군 미탄면,1563.0,919.0,1.70,1.19,미탄면
3,방림복지회관,연중무휴,9:00,18:00,9:00,18:00,Y,4.0,30000,None,...,128.394027,2022-11-30,방림면,강원특별자치도 평창군 방림면,강원특별자치도 평창군 방림면,2536.0,1530.0,3.32,2.21,방림면
4,계촌복지회관,연중무휴,9:00,18:00,9:00,18:00,Y,4.0,30000,None,...,128.306073,2022-11-30,방림면,강원특별자치도 평창군 방림면,강원특별자치도 평창군 방림면,2536.0,1530.0,3.32,2.21,방림면


In [35]:
# 병합을 위해 추가적으로 생성했던 컬럼들을 제거합니다
merged.drop(["동명칭_x", "동명칭_y", "key"], axis = 1, inplace = True)

# 최종적으로 병합된 데이터를 저장합니다.
# 이 데이터는 6.6. 장에서 다시 활용됩니다.
merged.to_excel("merged_clean.xlsx", index=False)